In [ ]:
import mlflow
from mlflow.tracking import client
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os,sys
sys.path.insert(1, os.path.join(os.getcwd()  , '../../src/gz21_ocean_momentum'))
from utils import select_experiment, select_run
from analysis.utils import plot_dataset

In [ ]:
test_exp_name = select_experiment()
test_exp = mlflow.get_experiment_by_name(test_exp_name)
test_exp_id = test_exp.experiment_id

In [ ]:
run = select_run(experiment_ids=test_exp_id)

In [ ]:
run

In [ ]:
client_ = client.MlflowClient()
data_file_name = client_.download_artifacts(run['params.data_run_id'], 'forcing')
print(data_file_name)
data = xr.open_zarr(data_file)

In [ ]:
data

In [ ]:
pred_file_name = client_.download_artifacts(run.run_id, 'test_output_0')
pred = xr.open_zarr(pred_file_name)

In [ ]:
merged = xr.merge((data, pred))

In [ ]:
merged

In [ ]:
plot_dataset(merged.isel(time=10), vmin=-1e-7, vmax=1e-7)